In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors


# 0.0 Load data

In [36]:
data = pd.read_csv('data/data_traduct.csv', index_col=0)
data.head(5)
data.sort_values('mode', ascending=False).head(10)

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
Unnamed: 0,,,,,,,,,,,,,,,,,,,,
113983,6X6wIzuxsh7GVNMPz1xTNa,Hillsong Worship,No Other Name,Depths,37,377697,False,0.3630,0.4620,7,-8.232,1,0.0284,0.008870,0.000009,0.7060,0.0687,80.003,4,música mundial
113980,64BdW5aHuB3C0QP3qjBJVi,I AM THEY,I AM THEY,Amen,41,238146,False,0.5160,0.6340,7,-4.850,1,0.0339,0.030000,0.000002,0.0813,0.0921,76.994,4,música mundial
113978,7IPetVJ4VQgCT5Sh5VA4h1,Lucas Cervetti,Trempülkalwe,Maremágnum,21,260506,False,0.4050,0.2830,4,-9.471,1,0.0300,0.894000,0.806000,0.1190,0.0813,99.947,4,música mundial
113977,3FuOBnk7Xsglp7AIsHQVaA,Reiki Armonía,Música Relajante para Reiki - Canciones de Fon...,Paz Mental,21,290999,False,0.0672,0.0903,9,-23.496,1,0.0401,0.970000,0.977000,0.0930,0.0361,76.042,4,música mundial
113976,11Jywidl4OVzmjziGR0QiO,Urban Rescue,Wild Heart,Alive In You,42,253546,False,0.5220,0.6970,2,-7.210,1,0.0344,0.055700,0.000000,0.1340,0.2880,75.972,4,música mundial
113974,0VVvjjtKyOeT2AEMTSQ33t,Chris Tomlin,And If Our God Is For Us...,Faithful,34,285120,False,0.6180,0.4190,6,-6.156,1,0.0279,0.309000,0.000000,0.1070,0.3730,131.973,4,música mundial
113973,1d2LyEJ1zZ0gWtlxyZQpDE,Vertical Worship,The Rock Won't Move,I Will Follow,44,260839,False,0.3190,0.5770,4,-7.679,1,0.0330,0.049900,0.000000,0.7290,0.1890,97.678,4,música mundial
113972,6xZuzUxNn5AGIPQ2FvXXLP,Mc_team,Frecuencias Curativas Solfeggio 528 Hz,528Hz La frecuencia del amor,21,170387,False,0.1700,0.0520,2,-28.693,1,0.0374,0.875000,0.904000,0.1290,0.0475,114.573,4,música mundial
113971,4aLBgBxa1ajNoWD3OSDay6,Ondas Delta,Abre Tu Mente: Música con Sonidos Binaurales y...,Ondas Cerebrales Delta,21,359003,False,0.3120,0.0330,9,-25.894,1,0.0307,0.853000,0.851000,0.1140,0.0639,79.655,4,música mundial


# 1.0 Data exploration

### Descrição das Colunas

- **track_id**: O ID do Spotify para a faixa
- **artists**: Os nomes dos artistas que interpretaram a faixa. Se houver mais de um artista, eles são separados por ponto e vírgula (;)
- **album_name**: O nome do álbum no qual a faixa aparece
- **track_name**: Nome da faixa
- **popularity**: A popularidade de uma faixa é um valor entre 0 e 100, sendo 100 a mais popular. A popularidade é calculada por algoritmo e é baseada, em grande parte, no número total de execuções da faixa e quão recentes essas execuções são. Em termos gerais, faixas que estão sendo muito tocadas agora terão maior popularidade do que faixas tocadas muito no passado. Faixas duplicadas (por exemplo, a mesma faixa de um single e de um álbum) são avaliadas separadamente. A popularidade do artista e do álbum é derivada matematicamente da popularidade das faixas.
- **duration_ms**: A duração da faixa em milissegundos
- **explicit**: Se a faixa contém ou não letras explícitas (true = sim, tem; false = não tem OU desconhecido)
- **danceability**: Danceability descreve o quão adequada uma faixa é para dançar, com base em uma combinação de elementos musicais, incluindo o tempo, a estabilidade do ritmo, a força do beat e a regularidade geral. Um valor de 0,0 é menos dançável e 1,0 é mais dançável
- **energy**: Energy é uma medida de 0,0 a 1,0 e representa uma medida perceptual de intensidade e atividade. Normalmente, faixas energéticas são rápidas, altas e barulhentas. Por exemplo, o death metal tem alta energia, enquanto uma preleção de Bach tem baixa energia
- **key**: A tonalidade em que a faixa está. Números inteiros mapeiam para tons usando a notação padrão de Pitch Class. Ex: 0 = C, 1 = C♯/D♭, 2 = D, e assim por diante. Se nenhuma tonalidade for detectada, o valor será -1
- **loudness**: O volume geral da faixa em decibéis (dB)
- **mode**: O modo indica a modalidade (maior ou menor) de uma faixa, o tipo de escala de onde o conteúdo melódico foi derivado. Maior é representado por 1 e menor por 0
- **speechiness**: Speechiness detecta a presença de palavras faladas em uma faixa. Quanto mais exclusivamente falada for a gravação (por exemplo, talk show, audiolivro, poesia), mais próximo de 1,0 será o valor do atributo. Valores acima de 0,66 descrevem faixas que provavelmente são feitas inteiramente de palavras faladas. Valores entre 0,33 e 0,66 descrevem faixas que podem conter tanto música quanto fala, seja em seções ou camadas, incluindo casos como música rap. Valores abaixo de 0,33 provavelmente representam música e outras faixas não relacionadas à fala
- **acousticness**: Uma medida de confiança de 0,0 a 1,0 de se a faixa é acústica. 1,0 representa alta confiança de que a faixa é acústica
- **instrumentalness**: Prediz se uma faixa não contém vocais. Sons como "ooh" e "aah" são tratados como instrumentais nesse contexto. Faixas de rap ou de palavra falada são claramente "vocais". Quanto mais próximo de 1,0 for o valor de instrumentalness, maior a probabilidade de que a faixa não contenha conteúdo vocal
- **liveness**: Detecta a presença de uma audiência na gravação. Valores mais altos de liveness indicam maior probabilidade de que a faixa tenha sido gravada ao vivo. Um valor acima de 0,8 oferece uma forte probabilidade de que a faixa seja ao vivo
- **valence**: Uma medida de 0,0 a 1,0 que descreve a positividade musical transmitida por uma faixa. Faixas com alta valência soam mais positivas (por exemplo, felizes, alegres, eufóricas), enquanto faixas com baixa valência soam mais negativas (por exemplo, tristes, deprimidas, zangadas)
- **tempo**: O tempo estimado geral de uma faixa em batidas por minuto (BPM). Em terminologia musical, o tempo é a velocidade ou o ritmo de uma peça dada e deriva diretamente da duração média das batidas
- **time_signature**: Uma assinatura de tempo estimada. A assinatura de tempo (métrica) é uma convenção de notação para especificar quantas batidas existem em cada compasso (ou medida). A assinatura de tempo varia de 3 a 7, indicando assinaturas de tempo de 3/4 a 7/4
- **track_genre**: O gênero ao qual a faixa pertence


Danceability
Energy
loudness 
mode 
valence
tempo 
acousticness
instrumentalness / speechiness

## 1.1 Estatísticas

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114000 entries, 0 to 113999
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   track_id          114000 non-null  object 
 1   artists           113999 non-null  object 
 2   album_name        113999 non-null  object 
 3   track_name        113999 non-null  object 
 4   popularity        114000 non-null  int64  
 5   duration_ms       114000 non-null  int64  
 6   explicit          114000 non-null  bool   
 7   danceability      114000 non-null  float64
 8   energy            114000 non-null  float64
 9   key               114000 non-null  int64  
 10  loudness          114000 non-null  float64
 11  mode              114000 non-null  int64  
 12  speechiness       114000 non-null  float64
 13  acousticness      114000 non-null  float64
 14  instrumentalness  114000 non-null  float64
 15  liveness          114000 non-null  float64
 16  valence           114000 

In [5]:
print("Número de linhas e colunas:", data.shape)

Número de linhas e colunas: (114000, 20)


In [6]:
data.describe()

,popularity,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,114000.000000,1.140000e+05,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000,114000.000000
mean,33.238535,2.280292e+05,0.566800,0.641383,5.309140,-8.258960,0.637553,0.084652,0.314910,0.156050,0.213553,0.474068,122.147837,3.904035
std,22.305078,1.072977e+05,0.173542,0.251529,3.559987,5.029337,0.480709,0.105732,0.332523,0.309555,0.190378,0.259261,29.978197,0.432621
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000,-49.531000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,17.000000,1.740660e+05,0.456000,0.472000,2.000000,-10.013000,0.000000,0.035900,0.016900,0.000000,0.098000,0.260000,99.218750,4.000000
50%,35.000000,2.129060e+05,0.580000,0.685000,5.000000,-7.004000,1.000000,0.048900,0.169000,0.000042,0.132000,0.464000,122.017000,4.000000
75%,50.000000,2.615060e+05,0.695000,0.854000,8.000000,-5.003000,1.000000,0.084500,0.598000,0.049000,0.273000,0.683000,140.071000,4.000000
max,100.000000,5.237295e+06,0.985000,1.000000,11.000000,4.532000,1.000000,0.965000,0.996000,1.000000,1.000000,0.995000,243.372000,5.000000


In [4]:
data['key'].unique()

array([ 1,  0,  2,  6, 11,  8,  4,  7,  3, 10,  5,  9])

In [30]:
data['track_genre'].unique()

array(['acústico', 'afrobeat', 'rock alternativo', 'alternativo',
       'ambiente', 'anime', 'black metal', 'bluegrass', 'blues',
       'brasileiro', 'breakbeat', 'britânico', 'cantopop',
       'house de Chicago', 'infantil', 'calmo', 'clássico', 'clube',
       'comédia', 'country', 'dança', 'dancehall', 'death metal',
       'deep house', 'techno de Detroit', 'disco', 'Disney',
       'drum and bass', 'dub', 'dubstep', 'EDM', 'eletro', 'eletrônico',
       'emo', 'folk', 'forró', 'francês', 'funk', 'garage', 'alemão',
       'gospel', 'gótico', 'grindcore', 'groove', 'grunge', 'guitarra',
       'feliz', 'hard rock', 'hardcore', 'hardstyle', 'heavy metal',
       'hip-hop', 'honky-tonk', 'house', 'IDM', 'indiano', 'indie pop',
       'indie', 'industrial', 'iraniano', 'J-dance', 'J-idol', 'J-pop',
       'J-rock', 'jazz', 'K-pop', 'latino', 'malaio', 'mandopop', 'metal',
       'metalcore', 'minimal techno', 'MPB', 'nova era', 'ópera',
       'pagode', 'festa', 'piano', 'pop de fi

## 1.2 Removendo Duplicatas

In [8]:
df1 = data.drop_duplicates(subset=['artists', 'track_name'], keep='first')
print("Número de linhas antes de remover duplicatas:", len(data))
print("Número de linhas após remover duplicatas:", len(df1))

Número de linhas antes de remover duplicatas: 114000
Número de linhas após remover duplicatas: 81344


# 2.0 Preprocessing

In [9]:
df2 = df1.copy()
df2.head(2)

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,1,-6.746,0,0.1430,0.0322,0.000001,0.358,0.715,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.166,1,-17.235,1,0.0763,0.9240,0.000006,0.101,0.267,77.489,4,acoustic


In [10]:

# Selecionar atributos de áudio
features = ['artists','track_name','danceability', 'energy', 'valence', 'acousticness', 
            'instrumentalness', 'liveness', 'loudness', 'speechiness']

available_features = [f for f in features if f in data.columns]
print("Atributos disponíveis:", available_features)

# Preparar os dados
data_features = df2[available_features]
scaler = StandardScaler()


Atributos disponíveis: ['artists', 'track_name', 'danceability', 'energy', 'valence', 'acousticness', 'instrumentalness', 'liveness', 'loudness', 'speechiness']


# 3.0 Modelagem

In [11]:
df3 = data_features.copy()

## 3.1  Train Test Split

In [12]:
# # Dividir em treino e teste
# train_data, test_data = train_test_split(df3, test_size=0.2, random_state=42)
# train_scaled = scaler.fit_transform(train_data[available_features].drop(columns=['artists', 'track_name']))
# test_scaled = scaler.transform(test_data[available_features].drop(columns=['artists', 'track_name']))

# print("Shape do conjunto de treino:", train_scaled.shape)  
# print("Shape do conjunto de teste:", test_scaled.shape)  

In [24]:
# Configurar o modelo KNN
knn = NearestNeighbors(n_neighbors=6, metric='euclidean')  # k=6 para pegar 5 vizinhos + o centroide
knn.fit(df3.drop(columns=['artists', 'track_name']))
print("Modelo KNN treinado no conjunto de treino!")

Modelo KNN treinado no conjunto de treino!


In [19]:
def recomendar_musicas_knn_unica(user_musica_name, data, knn_model,top_n=5):
    # Encontrar a linha correspondente ao user_musica_name
    idx = data.index[data['track_name'] == user_musica_name].tolist()[0]
    track_features = data.loc[idx, :].values.reshape(1, -1)[:, 2:]
    track_features = pd.DataFrame(track_features, columns=data.columns[2:])

    # Encontrar os vizinhos mais próximos
    distances, indices = knn_model.kneighbors(track_features)
    top_indices = indices[0][:top_n]  # Pegamos os top_n diretamente
    return data.iloc[top_indices]

n=0

# Testar com uma música específica
user_musica_name = df3['track_name'].iloc[n]  
print("\nRecomendações para a música:", user_musica_name,' - ', df3['artists'].iloc[n] ,'\n')

recomendacoes = recomendar_musicas_knn_unica(user_musica_name,df3, knn)
recomendacoes[['track_name', 'artists']]


Recomendações para a música: Comedy  -  Gen Hoshino 



,track_name,artists
0,Comedy,Gen Hoshino
32700,ili,TroyBoi
111986,Anything U Want (feat. QBall & Curt Cazal),Rae & Christian;Q*Ball;Curt Cazal
62314,好きだから。,『ユイカ』;れん
60723,空ノムコウ,Maharajan


In [20]:
recomendacoes

,artists,track_name,danceability,energy,valence,acousticness,instrumentalness,liveness,loudness,speechiness
0,Gen Hoshino,Comedy,0.676,0.461,0.715,0.0322,0.000001,0.358,-6.746,0.1430
32700,TroyBoi,ili,0.634,0.588,0.770,0.0575,0.002310,0.441,-6.753,0.1060
111986,Rae & Christian;Q*Ball;Curt Cazal,Anything U Want (feat. QBall & Curt Cazal),0.753,0.562,0.600,0.0270,0.024500,0.274,-6.797,0.1760
62314,『ユイカ』;れん,好きだから。,0.603,0.481,0.698,0.1730,0.000000,0.329,-6.689,0.0345
60723,Maharajan,空ノムコウ,0.597,0.599,0.657,0.0133,0.000009,0.278,-6.729,0.0442


# Visualizando K-NN 

In [21]:
import plotly.graph_objects as go

n_aleatorio = 20
fig = go.Figure(data=[
    go.Scatter(x=[recomendacoes['danceability'].iloc[0]], y=[recomendacoes['energy'].iloc[0]], 
               mode='markers', marker=dict(size=12, color='red'), name='Musica do usuário'),

    go.Scatter(x=recomendacoes['danceability'].iloc[1:], y=recomendacoes['energy'].iloc[1:], 
               mode='markers', marker=dict(size=10, color='blue'), name='Recomendações'),

    go.Scatter(x=df3.sample(n_aleatorio, random_state=42)['danceability'], y=df3.sample(n_aleatorio, random_state=42)['energy'], 
               mode='markers', marker=dict(size=8, color='green'), name='Pontos Aleatórios')
])

fig.update_layout(
    xaxis_title='Danceability',
    yaxis_title='Energy',
    showlegend=True
)

fig.show()

In [17]:
import plotly.graph_objects as go

# Primeiro ponto (vermelho) e os outros 4 (azul) das recomendações
fig = go.Figure(data=[
    go.Scatter3d(x=[recomendacoes['danceability'].iloc[0]], y=[recomendacoes['energy'].iloc[0]], z=[recomendacoes['instrumentalness'].iloc[0]], 
                 mode='markers', marker=dict(size=8, color='red'), text=[recomendacoes['track_name'].iloc[0]], name = 'Música selecionada'),

    go.Scatter3d(x=recomendacoes['danceability'].iloc[1:], y=recomendacoes['energy'].iloc[1:], z=recomendacoes['instrumentalness'].iloc[1:], 
                 mode='markers', marker=dict(size=6, color='blue'), text=recomendacoes['track_name'].iloc[1:], name = 'Recomendações'),

    go.Scatter3d(x=df3.sample(50, random_state=42)['danceability'], y=df3.sample(50, random_state=42)['energy'], z=df3.sample(50, random_state=42)['instrumentalness'], 
                 mode='markers', marker=dict(size=4, color='green'), text=df3.sample(50, random_state=42)['track_name'], name = 'Outras músicas')
])

fig.update_layout(scene=dict(xaxis_title='Danceability', yaxis_title='Energy', zaxis_title='Valence'))

fig.show()
